In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from xgboost import XGBRegressor


In [22]:

data= pd.read_csv("../input/razor-sample-data/Razor_sample.csv")

In [23]:
print(data)

In [24]:
data.head
import pandas_profiling


In [25]:
#INITIALLY DROPPING THE FEATURES WHICH LOGICALLY I DONT FIND RELEVENT IN TERMS OF DETERMINING THE BEST SELLER.
#SO WE USE DROP FN TO GET RID OF THOSE COLUMNS FROM OUR DATASET.

data = data.drop(['Date Added', 'category','sellerlink', 'sellerlink-url', 'sellerstorefront-url', 'sellerdetails','seller business name', 'businessaddress','Sample Brand URL','Unnamed: 18','Unnamed: 19', 'Unnamed: 20',"Sample brand name"], axis=1)

In [26]:
#NOW AFTER GETTING RID OF THE USELESS FEATURES, I WANT TO GET AN DETAILED OVERALL ANALYSIS OF THESE FEATURES AND 
#THEIR CORRELATION TO EACH OTHER.
#SO FOR THAT TO AVOID THE MANUAL LABOUR, I USE PROFILE REPORT FUNCTION OF PANDAS, WHICH GIVES ME A DETAILED ANALYSIS.
data.profile_report()


In [27]:
#Since AFTER ANALYSING THE DATA, I saw there were many sellers with incomplete info(features) we drop those sellers from out data.
#It is important so that it doesnt complicate our process.

data=data.dropna()

In [28]:
#ALSO WE CAN SEE THAT BY ANALYSING THE VARIOUS COEFFICIENT MATRIX WHICH TELLS US ABOUT HOW CLOSELY OUR FEATURES ARE
#INTERRELATED TO EACH OTHER. WE CAN SEE hero product 1 and 2 ARE HIGHLY CORRELATED WITH INDEX OF MORE THAN 0.7.
#SINCE hero product 1 and 2 are highly correlated we drop one of them. So logically I dropped hero product 2, Since hero
#product 1 is more important determining factor.


data = data.drop(['Hero Product 2 #ratings'],axis =1)
print(data)



In [ ]:
#SINCE THE "sellerproductcount" IS IN STRING FORMAT WE EXTRACT THE NUMBERS WHICH I THINK WILL BE USEFUL, WHICH
# IS BASICALLY THE COUNT OF PRODUCT.



In [33]:
def get_product_cnt(string):
    li = string.split(' ')
    s = li[-2]
    l1 = s.split(',')
    t=""
    for x in l1:
        t = t+ x
    
    return int(t)
print(data['sellerproductcount'])

In [34]:
data['sellerproductcount'] = data['sellerproductcount'].apply(get_product_cnt)



In [35]:
print(data['sellerproductcount'])

In [36]:
# SINCE THE "SELLERRATING FEATURE" IS IN STRING FORMAT WE EXTRACT THE NUMBERS WHICH I THINK WILL BE USEFUL 
# SO BASICALLY ONLY EXTRACTING THE COUNT OF POSITIVE RATING AND % OF POSITIVE RATING.


In [37]:
# FUNCTION FOR % OF POSITIVE RATING.

def get_percent(string):
    li = string.split('%')
    s = li[0]    
    return int(s)
li = []
l2 = data['sellerratings']
print(l2)

In [38]:

data['sellerratings'] = data['sellerratings'].apply(get_percent)
data['sellerratings']

In [39]:
# FUNCTION FOR  COUNT OF POSITIVE RATING.

def get_rating(string):
    l1 = string.split(' ')
    s = l1[-2]
    if(s == "total"):
        s = l1[-3]
        
    t = s[1:]
    return int(t)

print(l2)
data

In [40]:
l2 = l2.apply(get_rating)
print(l2)
data.rename(columns = {"sellerratings":"% of +ve ratings"},inplace=True)

In [41]:
#ALSO WE NEED TO ADD A NEW FEATURE SINCE "SELLERRATING FEATURE" GIVES US TWO SEPERATE FEATURE AS WE CAN SEE ABOVE.

data = (pd.concat([data,l2],axis=1))

In [42]:
# NOW WE GET OUR FINAL FEATURES WHICH IS IMPORTANT FOR OUR PROCESS.
data

In [43]:

data.profile_report()


In [ ]:
# SINCE TO AVOID MANUAL SELECTION, I DECIDED WE CAN DO K-MEANS CLUSTERING (UNSUPERVISED LEARNING) TO GET THE
# REQUIRED GROUP OF BEST SELLERS SINCE THEY WILL POSSESS SIMILAR ATTRIBUTES.


In [44]:
#BEFORE K MEANS WE APPLY PCA FOR EASY DATA VISUALISATION AND WHICH REDUCES THE NOISY DATASET AND IMPROVE THE ACCURACY
#OF THE CLUSTERING PROCESS. 
#IT IS IMPORTANT TO SCALE THE DATASET BEFORE PCA.

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
scalar = StandardScaler()
  
# fitting
scalar.fit(data)
scaled_data = scalar.transform(data)
  
# Importing PCA
from sklearn.decomposition import PCA
  
# Let's say, components = 2
pca = PCA(n_components = 2)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)
  
print(x_pca.shape)
x_pca

In [47]:
# PLOTTING THE GRAPH
  
plt.scatter(x_pca[:, 0], x_pca[:, 1], cmap ='plasma')
  
# labeling x and y axes
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')

In [64]:
plt.scatter(x_pca[0, 0], x_pca[0, 1], cmap ='plasma')

In [46]:
# APPLYING KMEANS CLUSTERING METHOD

from sklearn.cluster import KMeans
%matplotlib inline
Kmean = KMeans(n_clusters=3)
Kmean.fit(x_pca)
Kmean.cluster_centers_
plt.scatter(x_pca[ : ,0], x_pca[ : , 1])

k1 = Kmean.labels_

print(k1)
df = pd.DataFrame(k1)
print(df)

In [49]:
k1
# SINCE WE CAN OBSERVE THAT OUR SELLER AT THE 0TH INDEX FROM OUR DATA, WE CAN CONCLUDE THAT OUR 1ST SELLER
# MUST BE ONE OF THE BEST SELLERS AMONG THE WHOLE DATASET. 
# SO THE GROUP IN WHICH THAT SELLER WILL BELONG WILL BE ONE OF THE BEST SELLERS.


In [50]:
data


In [ ]:
# SO WE CAN SEE THE SELLER IN OUR 0TH INDEX BELONGS TO GROUP 2, SO OUR DESIRED GROUP WILL BE GROUP 2.
# SO WE EXTRACT ALL THE SELLERS IN OUR GROUP 2 AND PUT IT IN A LIST.

In [73]:
df.value_counts()
# WE CAN SEE OUR DESIRED GROUP HAS 43 SELLERS.

In [59]:
print(data.index)
print(k1[0])

In [65]:
Final_sellers = []
cnt = 0
for index1 in data.index :
    if(k1[cnt]==2):
        Final_sellers.append("seller " +str(index1+1))
    cnt=cnt+1
    

In [66]:
#final best sellers list
Final_sellers

In [64]:
len(Final_sellers)